In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn import datasets, linear_model # 引用 sklearn库，主要为了使用其中的线性回归模块

# 创建数据集，把数据写入到numpy数组
import numpy as np  # 引用numpy库，主要用来做科学计算
import matplotlib.pyplot as plt   # 引用matplotlib库，主要用来画图


df_show = pd.read_excel('data/1029_show.xls',index_col=0)
df_click = pd.read_excel('data/1029_click.xls',index_col=0)
print(df_show.shape)
print(df_click.shape)
df = pd.merge(df_show, df_click, how='left', on=['_source/uid'])
def f(row):
    if pd.isnull(row['_source/ua_y']):
        val = 0
    else:
        val = 1
    return val

df['click'] = df.apply(f, axis=1)
df = df[['_source/cTime_x','_source/uid',
         '_source/appkey_x','_source/adSpaceKey_x','_source/advertiserId','_source/advId_x',
         '_source/gameHtmlId_x','_source/extData/adTriggerType_x',
         '_source/clientProvince_x','_source/clientCity_x',
         '_source/uaBean/osName_x','_source/uaBean/brand_x','_source/uaBean/model_x','_source/uaBean/browserName_x',
         'click']]
print(df)
df.to_excel('dataset.xls',encoding='utf_8')


(367, 66)
(137, 70)
                    _source/cTime_x         _source/uid  \
0    29/Oct/2019:13:00:25:458 +0800  638723809549094016   
1    29/Oct/2019:13:00:25:460 +0800  638723753957789056   
2    29/Oct/2019:13:00:25:556 +0800  637935474463751936   
3    29/Oct/2019:13:00:25:072 +0800  638723680070930048   
4    29/Oct/2019:13:00:24:869 +0800  638723013592099968   
5    29/Oct/2019:13:00:24:864 +0800  638723774564674944   
6    29/Oct/2019:13:00:25:094 +0800  638723745808257024   
7    29/Oct/2019:13:00:25:017 +0800  638723682717805952   
8    29/Oct/2019:13:00:25:818 +0800  638723813454262016   
9    29/Oct/2019:13:00:25:282 +0800  637358848348797952   
10   29/Oct/2019:13:00:25:464 +0800  638449324380269952   
11   29/Oct/2019:13:00:25:449 +0800  638722407687273984   
12   29/Oct/2019:13:00:25:602 +0800  638721219285762048   
13   29/Oct/2019:13:00:25:516 +0800  638285135472898048   
14   29/Oct/2019:13:00:25:351 +0800  638722612444672000   
15   29/Oct/2019:13:00:25:665 +0800 

In [ ]:
import numpy as np
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score
from sklearn.preprocessing.data import OneHotEncoder

class GradientBoostingWithLR(object):
    def __init__(self):
        self.gbdt_model = None
        self.lr_model = None
        self.gbdt_encoder = None
        self.X_train_leafs = None
        self.X_test_leafs = None
        self.X_trans = None

    def gbdt_train(self, X_train, y_train):
        """定义GBDT模型
        """
        gbdt_model = GradientBoostingClassifier(n_estimators=10, 
                                          max_depth=6, 
                                          verbose=0,
                                          max_features=0.5)
        # 训练学习
        gbdt_model.fit(X_train, y_train)
        return gbdt_model

    def lr_train(self, X_train, y_train):
        """定义LR模型
        """
        lr_model = LogisticRegression()
        lr_model.fit(X_train, y_train)    # 预测及AUC评测
        return lr_model
    

    def gbdt_lr_train(self,X_train, y_train,X_test):
        """训练gbdt+lr模型
        """
        self.gbdt_model = self.gbdt_train(X_train, y_train)

        # 使用GBDT的apply方法对原有特征进行编码
        self.X_train_leafs = self.gbdt_model.apply(X_train)[:,:,0]
        
        # 对特征进行ont-hot编码
        self.gbdt_encoder = OneHotEncoder(categories='auto')
        self.gbdt_encoder.fit(self.X_train_leafs)
        self.X_trans = self.gbdt_encoder.fit_transform(self.X_train_leafs)
        
        #采用LR进行训练
        self.lr_model = self.lr_train(self.X_trans, y_train)
        return self.lr_model
    
    def gbdt_lr_pred(self, model, X_test, y_test):
        """预测及AUC评估
        """
        self.X_test_leafs = self.gbdt_model.apply(X_test)[:,:,0]
        
        (train_rows, cols) =self.X_train_leafs.shape
        X_trans_all = self.gbdt_encoder.fit_transform(np.concatenate((self.X_train_leafs, self.X_test_leafs), axis=0))
        
        y_pred = model.predict_proba(X_trans_all[train_rows:])[:, 1]
        auc_score = roc_auc_score(y_test, y_pred)
        print('GBDT+LR AUC score: %.5f' % auc_score)
        return auc_score
    
    def model_assessment(self, model, X_test, y_test, model_name="GBDT"):
        """模型评估
        """
        y_pred = model.predict_proba(X_test)[:,1]
        auc_score = roc_auc_score(y_test, y_pred)
        print("%s AUC score: %.5f" % (model_name,auc_score))
        return auc_score

In [14]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

def load_data():
    """
    调用sklearn的iris数据集，将多类数据构造成2分类数据,同时切分训练测试数据集
    """
    iris_data = load_iris()
    print(type(iris_data))
    print(iris_data)
    X = iris_data['data']
    y = iris_data['target'] == 2
    
    print(X.shape)
    print(y.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
    print(X_train.shape)
    print(X_test.shape)
    print(y_train.shape)
    print(y_test.shape)
    
    return X_train,X_test,y_train, y_test

X_train,X_test,y_train, y_test = load_data()

        
gblr = GradientBoostingWithLR()
gbdt_lr_model = gblr.gbdt_lr_train(X_train, y_train, X_test)
gblr.model_assessment(gblr.gbdt_model, X_test, y_test)
gblr.gbdt_lr_pred(gbdt_lr_model, X_test, y_test)

<class 'sklearn.utils.Bunch'>
{'data': array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
    

/Users/baicongmin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9945054945054945

In [12]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

# save load_iris() sklearn dataset to iris
# if you'd like to check dataset type use: type(load_iris())
# if you'd like to view list of attributes use: dir(load_iris())
iris = load_iris()

# np.c_ is the numpy concatenate function
# which is used to concat iris['data'] and iris['target'] arrays 
# for pandas column argument: concat iris['feature_names'] list
# and string list (in this case one string); you can make this anything you'd like..  
# the original dataset would probably call this ['Species']
data1 = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
print(data1)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
5                  5.4               3.9                1.7               0.4   
6                  4.6               3.4                1.4               0.3   
7                  5.0               3.4                1.5               0.2   
8                  4.4               2.9                1.4               0.2   
9                  4.9               3.1                1.5               0.1   
10                 5.4               3.7                1.5               0.2   
11                 4.8      

In [13]:
import numpy as np
import sklearn.datasets

examples = []
examples.append('some text')
examples.append('another example text')
examples.append('example 3')

target = np.zeros((3,), dtype=np.int64)
target[0] = 0
target[1] = 1
target[2] = 0
dataset = sklearn.datasets.base.Bunch(data=examples, target=target)
print(dataset)

{'data': ['some text', 'another example text', 'example 3'], 'target': array([0, 1, 0])}
